# Crawling data from box office webpage and load to AWS Mysql Server

In [1]:
import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup
import ssl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import re

In [2]:
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [3]:
pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


In [4]:
import mysql.connector

In [5]:
####create function to get web link to 2015-2019 boxoffice Domestic Yearly Gross table####

In [5]:
def get_url(year):
    url = 'https://www.boxofficemojo.com/year/'+str(year)+'/?grossesOption=calendarGrosses'
    return url

In [7]:
####create function to scrape data from box-office

In [6]:
def main_table(url):  

  html = urllib.request.urlopen(url, context=ctx).read()
  soup = BeautifulSoup(html, 'html.parser')

  t1=soup.find_all('table')

  t2=t1[0].find_all('tr')

  row=[]
  for i in range(1,len(t2)):
    row.append(t2[i].find_all('td'))

  t3=[]
  for i in range(len(row)):
    test=[]
    for j in row[i]:
      test.append(j.get_text())
    t3.append(test)

  main_info=[]
  for i in t3:
    r=[]
    for j in [0,1,5,6,7,8]:
      r.append(i[j])
    main_info.append(r)

  return main_info

In [12]:
#### extract web link to detail info of each movie ####

In [13]:
def detail_link(url):
  html = urllib.request.urlopen(url, context=ctx).read()
  soup = BeautifulSoup(html, 'html.parser')
  tags = soup('a')
  lk =[]
  for tag in tags:
    i=tag.get('href',None)
    if i.startswith('/release'):
      lk.append(i)
  return lk

In [14]:
detail_lk_2019=detail_link(lk2019)

In [15]:
#### crawling detail info of each movie, like "genre", from box office 

In [16]:
def detail(url):
  url_link1='https://www.boxofficemojo.com'+url
  html2 = urllib.request.urlopen(url_link1, context=ctx).read()
  soup2 = BeautifulSoup(html2, 'html.parser')

  table_id=re.findall('/release/(r.+)/',url)

  a1=soup2.find_all('main')
  a2=a1[0].find_all('div',attrs={'class':'a-section a-spacing-none mojo-performance-summary-table'})
  a3=a2[0].find_all('span')
  test2=[i.get_text(strip=True) for i in a3]
  row_1={}
  #dic_index=['Domestic%','Dome_revenue','International%','Inte_revenue','Total_revnue']
  for i in range(len(test2)):
    if test2[i].startswith('Domestic'):
      row_1['Domestic%']=test2[i+1]
      row_1['Dome_revenue']=test2[i+2]
    elif test2[i].startswith('International'):
      row_1['International%']=test2[i+1]
      row_1['Inte_revenue']=test2[i+2]
    elif test2[i].startswith('World'):
      row_1['Total_revnue']=test2[i+1]
    else:
      continue
  test3=list(row_1.values())
    
  c1=soup2.find_all('div',attrs={'class':'a-section a-spacing-none mojo-summary-values mojo-hidden-from-mobile'})
  c2=c1[0].find_all('span')
  c3=[i.get_text(strip=';').strip() for i in c2]
  dic_index2=['Distributor','Budget','Release_date','Duration','Genre','In_release','Widest_release']
  row_3={}
  for i in range(len(c3)):
    if c3[i].startswith('Distributor'):
      row_3['Distributor']=c3[i+1]
    elif c3[i].startswith('Budget'):
      row_3['Budget']=c3[i+1]
    elif c3[i].startswith('Release Date'):
      row_3['Release_date']=c3[i+1]
    elif c3[i].startswith('Running Time'):
      row_3['Duration']=c3[i+1]
    elif c3[i].startswith('Genres'):
      row_3['Genre']=c3[i+1].split('\n    \n        ')
    elif c3[i].startswith('In Release'):
      row_3['In_release']=c3[i+1].split('/')
    elif c3[i].startswith('Widest Release'):
      row_3['Widest_release']=c3[i+1]
    else:
      continue

  keys=list(row_3.keys())
  if 'Budget' not in keys:
    row_3['Budget']='-'
  if 'Duration' not in keys:
    row_3['Duration']='-'
  if 'Genre' not in keys:
    row_3['Genre']='-'
  if 'Widest_release' not in keys:
    row_3['Widest_release']='-'

  c4=[]
  for i in dic_index2:
    val=row_3[i]
    c4.append(val)

  final=table_id+test3+c4
  return final

In [14]:
#### connect to AWS Mysql Server and create table for main_info of 2015 to 2019 and details of each movie ####

In [11]:
mycursor = mydb.cursor(buffered=True)

In [12]:
mycursor.execute("DROP TABLE IF EXISTS maininfo")
mycursor.execute("CREATE TABLE maininfo (rank_id VARCHAR(255), release_name VARCHAR(255),gross VARCHAR(255),theater VARCHAR(255),total_gross VARCHAR(255),release_date VARCHAR(255))")
#mycursor.execute("SHOW TABLES")

In [13]:
def maininfo_sql(main_info):
  for i in main_info:
    sql= "INSERT INTO maininfo (rank_id,release_name,gross,theater,total_gross,release_date) VALUES (%s,%s,%s,%s,%s,%s)"
    val=tuple(i)
    mycursor.execute(sql, val)
  
  mydb.commit()

In [14]:
maininfo_sql(main_info2019)

In [ ]:
mycursor.execute("DROP TABLE IF EXISTS details")
mycursor.execute("CREATE TABLE details (table_id VARCHAR(255), Domestic_percent VARCHAR(255),Dome_revenue VARCHAR(255),International_percent VARCHAR(255),Inte_revenue VARCHAR(255),Total_revnue VARCHAR(255),Company VARCHAR(255),Budget VARCHAR(255),Release_date VARCHAR(255),Duration VARCHAR(255),Genre VARCHAR(255),In_release VARCHAR(255),Widest_release VARCHAR(255))")    
mycursor.execute("SHOW TABLES")

In [ ]:
def details_sql(detail_final):
  for i in detail_final:
    new=[]
    for j in i:
      if type(j) is str:
        new.append(j)
      elif type(i) is list:
        k=','.join(j)
        new.append(k)
    sql= "INSERT INTO details (table_id, Domestic_percent,Dome_revenue,International_percent,Inte_revenue,Total_revnue,Company,Budget,Release_date,Duration,Genre,In_release,Widest_release) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
    val=tuple(new)
    mycursor.execute(sql, val)

  mydb.commit()

In [ ]:
mydb.close